In [25]:
import tensorflow as tf
#import tensorflow_federated as tff
from tensorflow.keras.applications import VGG16, MobileNetV3Small, MobileNetV2
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
IMAGE_SIZE=256

In [3]:
data_augmentation = keras.Sequential(
[layers.RandomFlip("horizontal"), layers.RandomCrop(IMAGE_SIZE, IMAGE_SIZE), layers.RandomRotation(0.1)]
)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-05-06 15:31:57.578585: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-06 15:31:57.578858: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
base_model = MobileNetV3Small(weights='imagenet', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False)
#base_model = MobileNetV2(weights='imagenet', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False)
base_model.trainable = False

In [17]:
import glob, cv2
import numpy as np

files = []
files.extend(glob.glob("dataset/client1/*.bmp"))

images = np.array([cv2.resize(cv2.imread(file),(IMAGE_SIZE, IMAGE_SIZE)) for file in files])
label= np.ones((len(images), 1))

In [19]:
images.shape
label.shape

(15, 1)

In [24]:
input = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
#x = data_augmentation(input)

x = keras.layers.Rescaling(scale=1/127.5, offset=-1)(input)

x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
output = keras.layers.Dense(1)(x)
model = keras.Model(input, output)

model.summary()

model.compile(
    optimizer=keras.optimizers.Nadam(),
    loss=keras.losses.BinaryFocalCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()]
)

model.fit(images, label, epochs=20)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 rescaling_6 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 MobilenetV3small (Functiona  (None, 8, 8, 576)        939120    
 l)                                                              
                                                                 
 global_average_pooling2d_2   (None, 576)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_2 (Dropout)         (None, 576)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 577 

2022-05-06 15:37:03.778756: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 4s 4s/step - loss: 0.2033 - binary_accuracy: 0.0000e+00
Epoch 2/20
1/1 [==============================] - 0s 30ms/step - loss: 0.1737 - binary_accuracy: 0.0000e+00
Epoch 3/20
1/1 [==============================] - 0s 23ms/step - loss: 0.1563 - binary_accuracy: 0.0000e+00
Epoch 4/20
1/1 [==============================] - 0s 25ms/step - loss: 0.1415 - binary_accuracy: 0.0000e+00
Epoch 5/20
1/1 [==============================] - 0s 25ms/step - loss: 0.1293 - binary_accuracy: 0.0667
Epoch 6/20
1/1 [==============================] - 0s 22ms/step - loss: 0.1166 - binary_accuracy: 0.0667
Epoch 7/20
1/1 [==============================] - 0s 24ms/step - loss: 0.1041 - binary_accuracy: 0.0667
Epoch 8/20
1/1 [==============================] - 0s 22ms/step - loss: 0.0943 - binary_accuracy: 0.2667
Epoch 9/20
1/1 [==============================] - 0s 21ms/step - loss: 0.0857 - binary_accuracy: 0.3333
Epoch 10/20
1/1 [==============================] - 0s 21ms/st